In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

In [ ]:
## Parameters
CAL_INPUT_IMAGE_DIR = "camera_cal"
CAL_OUTPUT_FILE = "calibration_params.p"
IMG_SIZE = (960, 1280)
COLOR_THRESHOLD = (170, 255)
GRADIENT_X_THRESHOLD = (20, 100)
GRADIENT_MAG_THRESHOLD = (20, 100)
GRADIENT_DIR_THRESHOLD =(0.7, 1.3)

In [ ]:
# Step 1: Calibrate camera, if not done before
class Utils:
    
    @staticmethod
    def calibrate_camera(image_dir, nx, ny, image_size, output_file):
        '''
        Compute and return camera calibration parameters
        '''
        
        if os.path.isfile(CAL_OUTPUT_FILE):
            print("Calibration has been done before, will skip!")
            cal_params = pickle.load( open( CAL_OUTPUT_FILE, "rb" ) )
            return cal_params
        else:    
            objp = np.zeros((ny*nx, 3), np.float32)
            objp[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2)
            objpoints = []
            imgpoints = []

            images = glob.glob(os.path.join(image_dir, '*.jpg'))
            for ids, fname in enumerate(images):
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
            if ret is True:
                objpoints.append(objp)
                imgpoints.append(corners)

            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
            calibration_params = {}
            calibration_params["mtx"] = mtx
            calibration_params["dist"] = dist
            f = open (CAL_OUTPUT_FILE, "wb")
            pickle.dump(calibration_params, f)
            print("Camera calibration parameters written to {}".format(CAL_OUTPUT_FILE))
            return calibration_params

    @statismethod
    def to_gray_scale(image):
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    @staticmethod
    def gaussian_blur(img, kernel_size):
        """Applies a Gaussian Noise kernel"""
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
    @staticmethod
    def canny(img, low_threshold, high_threshold):
        """Applies the Canny transform"""
        return cv2.Canny(img, low_threshold, high_threshold)

    @staticmethod
    def gradient_x_threshold(gray_image, min_max):
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        abs_sobelx = np.absolute(sobelx)
        scaled_sobelx = np.uint8(255 * (abs_sobelx/(np.max(abs_sobelx))))
        binary = np.zeros_like(scaled_sobelx)
        binary[(scaled_sobelx > min_max[0]) & (scaled_sobelx <= min_max[1])] = 1
        return binary
    
    @staticmethod
    def color_threshold(image, min_max):
        hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
        channel_s = hls[:,:,2]
        binary = np.zeros_like(channel_s)
        binary[(channel_s > min_max[0]) & (channel_s <=min_max[1])] = 1
        return binary
    
    @staticmethod
    def grad_dir_threshold(gray_image, min_max):
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
        angle = cv2.arctan(np.absolute(sobely), np.absolute(sobelx)
        binary =  np.zeros_like(angle)
        binary[(angle >= min_max[0]) & (angle <= min_max[1])] = 1
        return binary 

    @staticmethod                       
    def get_perspective_src_dst_points(sample_image):
        src_vertices = np.array([[(0.15 * x_size,y_size),(0.4 * x_size, 0.6 * y_size), (0.6 * x_size, 0.6 * y_size), (0.85 * x_size, y_size)]], dtype=np.int32)
        dst_vertices = np.array([[(0.15 * x_size,y_size),(0.15 * x_size, 0.6 * y_size), (0.85 * x_size, 0.6 * y_size), (0.85 * x_size, y_size)]], dtype=np.int32)
        return (src_vertices, dst_vertices)
            


In [ ]:
class Pipeline:
    
    def __init__():
        self.cal_mtx = None
        self.cal_dist = None
        self.pers_M = None
            
    def undistort(self, image):
        if self.cal_mtx is None:
            cal_params = Utils.calibrate_camera(CAL_INPUT_IMAGE_DIR, nx, ny)
            self.cal_mtx = cal_params["mtx"]
            self.cal_dist = cal_params["dist"]
        
        return cv2.undistort(image, mtx, dist, None, mtx)
        
    def apply_thresholds(self, image):
        gray = Utils.to_gray_scale(image)
        color_threshold_img    = Utils.color_threshold(image, COLOR_THRESHOLD)
        gradx_binary = Utils.gradient_x_threshold(gray, GRADIENT_X_THRESHOLD)
        mag_binary   = Utils.grad_mag_threshold(gray, GRADIENT_MAG_THRESHOLD)
        dir_binary   = Utils.grad_dir_threshold(gray, GRAD_DIR_THRESHOLD)
        combined = np.zeros_like(color_threshold_img)
        combined[(gradx_binary == 1) | ((mag_binary == 1) & (dir_binary == 1))] = 1
        return combined
    
    def warp_perspective(self, image_binary):
        if self.pers_M is None:
            src, dst = Utils.get_perspective_src_dst_points(image)
            self.pers_M = cv2.getPerspectiveTransform(src, dst)
        
        warped_binary = cv2.warpPerspective(image_binary, self.pers_M, image.shape, flags=cv2.INTER_LINEAR)
        return warped
    
    def find_lane_polynomials(self, warped_binary):
        # build histogram
        
        
        
        
        